In [1]:
import fceulib
from fceulib import VectorBytes
import numpy as np

In [2]:

START = 0x08
RUN = 0x02
JUMP = 0x01

def hold(mask, duration):
    return [mask for i in range(duration)]


In [3]:

emu = fceulib.runGame('mario.nes')
startInputs = hold(0x0, 120) + hold(START | JUMP, 30) + hold(0x0, 150)
jumpInputs = [hold(0x0, 5) + hold(JUMP, t) + hold(0x0, 300)
              for t in range(5, 300)]
for m in startInputs:
    emu.step(m, 0x0)
emu.stepFull(0x0, 0x0)
start = VectorBytes()
print("SAVE")
emu.save(start)

SAVE


In [4]:
def pointer_to_numpy(ptr):
    return np.array([ptr[xx] for xx in range(len(ptr))])

emu.load(start)
nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
nt = nt[:960].reshape(30,32)

In [5]:
import numpy as np
nt = np.array(nt)

import matplotlib.pyplot as plt
#plt.imshow(nt,interpolation='none')
#plt.show()

In [6]:
pt = pointer_to_numpy(emu.fc.ppu.PALRAM)
print pt

[34 41 26 15 34 54 23 15 34 48 33 15 34 39 23 15  0 22 39 24  0 26 48 39  0
 22 48 39  0 15 54 23]


In [ ]:
sprite_ram  = pointer_to_numpy(emu.fc.ppu.SPRAM)
def sprite_attributes_to_dict(attributes):
    masks = {'background' : 0x20,
        'hflip' : 0x40,
        'vflip'  : 0x80}
    sprite_attributes = {mask:attributes&masks[mask]>0 for mask in masks}
    sprite_attributes['palette'] = attributes & 0x03
    return sprite_attributes
to_plot = []
for ii in range(0,len(sprite_ram),4):
    print 'x: {}, y: {}, id:{}, attributes:{}'.format(sprite_ram[ii+3],
                                                      sprite_ram[ii],
                                                      sprite_ram[ii+1],
                                                      sprite_attributes_to_dict(sprite_ram[ii+2]))
    if sprite_ram[ii+1] > 0:
        to_plot.append([sprite_ram[ii+3],sprite_ram[ii]])
to_plot = np.array(to_plot,dtype='float')
print to_plot
plt.plot(to_plot[:,0]/8.,to_plot[:,1]/8.,'rx')
plt.imshow(nt,interpolation='none')
plt.show()


x: 88, y: 24, id:255, attributes:{'palette': 3, 'hflip': False, 'background': True, 'vflip': False}
x: 40, y: 176, id:252, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 48, y: 176, id:252, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 40, y: 184, id:252, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 48, y: 184, id:252, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 40, y: 192, id:58, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 48, y: 192, id:55, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 40, y: 200, id:79, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip': False}
x: 48, y: 200, id:79, attributes:{'palette': 0, 'hflip': True, 'background': False, 'vflip': False}
x: 0, y: 248, id:0, attributes:{'palette': 0, 'hflip': False, 'background': False, 'vflip

In [ ]:
help(fceulib)

Help on module fceulib:

NAME
    fceulib - Python wrapper for fceulib NES emulation

FILE
    /home/darkacloo/mechlearn/mechlearn/fceulib.so

CLASSES
    __builtin__.object
        BytePointer
        Cart
        Emulator
        FC
        FCEU
        PPU
        VectorBytes
        VectorShorts
    
    class BytePointer(__builtin__.object)
     |  Methods defined here:
     |  
     |  __getitem__(...)
     |      __getitem__(self: fceulib.BytePointer, arg0: int) -> int
     |  
     |  __init__(...)
     |      x.__init__(...) initializes x; see help(type(x)) for signature
     |  
     |  __len__(...)
     |      __len__(self: fceulib.BytePointer) -> int
     |  
     |  __setitem__(...)
     |      __setitem__(self: fceulib.BytePointer, arg0: int, arg1: float) -> None
     |  
     |  ----------------------------------------------------------------------
     |  Data and other attributes defined here:
     |  
     |  __new__ = <built-in method __new__ of type object>
     |  